In [ ]:
!wget -q --no-check-certificate https://www.crcv.ucf.edu/data/UCF101/UCF101.rar
!wget -q --no-check-certificate https://www.crcv.ucf.edu/data/UCF101/UCF101TrainTestSplits-RecognitionTask.zip

In [ ]:
%%capture
!unrar e /content/UCF101.rar /content/data/
!unzip -qq /content/UCF101TrainTestSplits-RecognitionTask.zip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from keras.layers import LSTM, Flatten, Bidirectional, Dense, Permute, multiply,Dropout, LayerNormalization,GaussianNoise, GaussianDropout ,GlobalAveragePooling1D
from keras import Model

In [ ]:
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import imageio

from imutils import paths
from tqdm import tqdm
import pandas as pd 
import numpy as np
import shutil
import cv2
import os

In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 20
EPOCHS = 60 
MAX_SEQ_LENGTH = 20
NUM_FEATURES = 512

In [ ]:
# Open the .txt file which have names of training videos
f = open("/content/ucfTrainTestlist/trainlist03.txt", "r")
temp = f.read()
videos = temp.split('\n')

# Create a dataframe having video names
train = pd.DataFrame()
train['video_name'] = videos
train = train[:-1]
train.head()

In [ ]:
# Open the .txt file which have names of test videos
with open("/content/ucfTrainTestlist/testlist03.txt", "r") as f:
    temp = f.read()
videos = temp.split("\n")

# Create a dataframe having video names
test = pd.DataFrame()
test["video_name"] = videos
test = test[:-1]
test.head()

,video_name
0,ApplyEyeMakeup/v_ApplyEyeMakeup_g15_c01.avi
1,ApplyEyeMakeup/v_ApplyEyeMakeup_g15_c02.avi
2,ApplyEyeMakeup/v_ApplyEyeMakeup_g15_c03.avi
3,ApplyEyeMakeup/v_ApplyEyeMakeup_g15_c04.avi
4,ApplyEyeMakeup/v_ApplyEyeMakeup_g15_c05.avi


In [ ]:
def extract_tag(video_path):
    return video_path.split("/")[0]

def separate_video_name(video_name):
    return video_name.split("/")[1]

def rectify_video_name(video_name):
    return video_name.split(" ")[0]

def move_videos(df, output_dir):
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    for i in tqdm(range(df.shape[0])):
        videoFile = df['video_name'][i].split("/")[-1]
        videoPath = os.path.join("data", videoFile)
        shutil.copy2(videoPath, output_dir)
    print()
    print(f"Total videos: {len(os.listdir(output_dir))}")

In [ ]:
train["tag"] = train["video_name"].apply(extract_tag)
train["video_name"] = train["video_name"].apply(separate_video_name)
train.head()

,video_name,tag
0,v_ApplyEyeMakeup_g01_c01.avi 1,ApplyEyeMakeup
1,v_ApplyEyeMakeup_g01_c02.avi 1,ApplyEyeMakeup
2,v_ApplyEyeMakeup_g01_c03.avi 1,ApplyEyeMakeup
3,v_ApplyEyeMakeup_g01_c04.avi 1,ApplyEyeMakeup
4,v_ApplyEyeMakeup_g01_c05.avi 1,ApplyEyeMakeup


In [ ]:
train["video_name"] = train["video_name"].apply(rectify_video_name)
train.head()

,video_name,tag
0,v_ApplyEyeMakeup_g01_c01.avi,ApplyEyeMakeup
1,v_ApplyEyeMakeup_g01_c02.avi,ApplyEyeMakeup
2,v_ApplyEyeMakeup_g01_c03.avi,ApplyEyeMakeup
3,v_ApplyEyeMakeup_g01_c04.avi,ApplyEyeMakeup
4,v_ApplyEyeMakeup_g01_c05.avi,ApplyEyeMakeup


In [ ]:
test["tag"] = test["video_name"].apply(extract_tag)
test["video_name"] = test["video_name"].apply(separate_video_name)
test.head()

,video_name,tag
0,v_ApplyEyeMakeup_g15_c01.avi,ApplyEyeMakeup
1,v_ApplyEyeMakeup_g15_c02.avi,ApplyEyeMakeup
2,v_ApplyEyeMakeup_g15_c03.avi,ApplyEyeMakeup
3,v_ApplyEyeMakeup_g15_c04.avi,ApplyEyeMakeup
4,v_ApplyEyeMakeup_g15_c05.avi,ApplyEyeMakeup


In [ ]:
n = 10
topNActs = train["tag"].value_counts().nlargest(n).reset_index()["index"].tolist()
train_new = train[train["tag"].isin(topNActs)]
test_new = test[test["tag"].isin(topNActs)]
train_new.shape, test_new.shape

((1181, 2), (445, 2))

In [ ]:
train_new = train_new.reset_index(drop=True)
test_new = test_new.reset_index(drop=True)

In [ ]:
move_videos(train_new, "/content/train2")
move_videos(test_new, "/content/test2")

100%|██████████| 1181/1181 [00:01<00:00, 659.54it/s]



Total videos: 1181


100%|██████████| 445/445 [00:01<00:00, 419.96it/s]


Total videos: 445


In [ ]:
train_new.to_csv("/content/train.csv", index=False)
test_new.to_csv("/content/test.csv", index=False)

In [ ]:
train_df = pd.read_csv("/content/train.csv")
test_df = pd.read_csv("/content/test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")

train_df.sample(10)

Total videos for training: 1181
Total videos for testing: 445


,video_name,tag
927,v_PlayingDhol_g23_c02.avi,PlayingDhol
896,v_PlayingDhol_g18_c02.avi,PlayingDhol
821,v_PlayingCello_g24_c07.avi,PlayingCello
364,v_Drumming_g02_c02.avi,Drumming
645,v_IceDancing_g16_c02.avi,IceDancing
767,v_PlayingCello_g16_c07.avi,PlayingCello
1134,v_TennisSwing_g18_c02.avi,TennisSwing
1143,v_TennisSwing_g19_c05.avi,TennisSwing
306,v_CricketShot_g17_c06.avi,CricketShot
414,v_Drumming_g16_c07.avi,Drumming


In [ ]:
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

In [ ]:
def build_feature_extractor():
    feature_extractor = keras.applications.VGG19(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.vgg19.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

80150528/80134624 [==============================] - 1s 0us/step


In [ ]:
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["tag"])
)
print(label_processor.get_vocabulary())

['BenchPress', 'BoxingPunchingBag', 'CricketShot', 'Drumming', 'HorseRiding', 'IceDancing', 'PlayingCello', 'PlayingDhol', 'ShavingBeard', 'TennisSwing']


In [ ]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    labels = df["tag"].values
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    
    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]
        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    
    return (frame_features, frame_masks), labels

train_data, train_labels  = prepare_all_videos(train_df, "train2")
test_data, test_labels = prepare_all_videos(test_df, "test2")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")

Frame features in train set: (1181, 20, 512)
Frame masks in train set: (1181, 20)


In [ ]:
def save_features():
  with open('/content/drive/MyDrive/ucf_07/train_data_ucf10.npy', 'wb') as f:
      np.save(f, train_data[0])
      np.save(f, train_data[1])
  with open('/content/drive/MyDrive/ucf_07/train_labels_ucf10.npy', 'wb') as f:
      np.save(f, train_labels)

  with open('/content/drive/MyDrive/ucf_07/test_data_ucf10.npy', 'wb') as f:
      np.save(f, test_data[0]) 
      np.save(f, test_data[1])
  with open('/content/drive/MyDrive/ucf_07/test_labels_ucf10.npy', 'wb') as f:
      np.save(f, test_labels)

save_features()
